In [68]:
import pandas as pd
import numpy as np
import torch as pt

import sklearn as sk
from sklearn import preprocessing
import tensorflow as tf

import matplotlib.pyplot as plot

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [69]:
climate_data = pd.read_csv('Data_climate_indices_V2.csv', index_col=0)

# Finding the overall training and testing data.

training = climate_data[1:361]
testing = climate_data[361:len(climate_data)]

# Splitting the x and y values for training and testing.

train_x = training[training.columns[1:training.shape[1]-1]]
train_y = training[training.columns[training.shape[1] -1:]].values[1:]

test_x = testing[testing.columns[1:testing.shape[1]-1]]
test_y = testing[testing.columns[testing.shape[1] -1:]].values[1:]

# Normalizing the training and testing variables.

min_max_scaler = preprocessing.MinMaxScaler()
train_x_scaled = min_max_scaler.fit_transform(train_x)
test_x_scaled = min_max_scaler.fit_transform(test_x)



In [80]:
# Creating the data with shifts for training.

logs = 15
shifted_x_train = []
shifted_y_train = []

for index in range(logs,train_x.shape[0] - 1):
    shifted_x_train.append(train_x_scaled[index - logs : index])
    shifted_y_train.append(train_y[index][0])

shifted_x_train, shifted_y_train = np.array(shifted_x_train), np.array(shifted_y_train)

print(shifted_x_train.shape)
print(shifted_y_train.shape)

(344, 15, 43)
(344,)


In [81]:
learning_model = Sequential()
learning_model.add(LSTM(units = 50, return_sequences = True, input_shape=(logs,train_x.shape[1])))
learning_model.add(Dropout(0.2))
learning_model.add(LSTM(units = 50))
learning_model.add(Dropout(0.2))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))
learning_model.add(Dense(4))


learning_model.compile(optimizer = 'adam' , loss = 'mean_squared_error')

In [82]:
learning_model.fit(shifted_x_train, shifted_y_train , epochs = 600 , batch_size = 12)

Epoch 1/600
29/29 [==============================] - 18s 23ms/step - loss: 12940.4912
Epoch 2/600
29/29 [==============================] - 1s 23ms/step - loss: 10873.1631
Epoch 3/600
29/29 [==============================] - 1s 24ms/step - loss: 8738.2725
Epoch 4/600
29/29 [==============================] - 1s 23ms/step - loss: 8209.0635
Epoch 5/600
29/29 [==============================] - 1s 26ms/step - loss: 8131.3394
Epoch 6/600
29/29 [==============================] - 1s 25ms/step - loss: 8054.3125
Epoch 7/600
29/29 [==============================] - 1s 24ms/step - loss: 7969.3955
Epoch 8/600
29/29 [==============================] - 1s 22ms/step - loss: 8050.9595
Epoch 9/600
29/29 [==============================] - 1s 24ms/step - loss: 8014.0908
Epoch 10/600
29/29 [==============================] - 1s 26ms/step - loss: 7925.7319
Epoch 11/600
29/29 [==============================] - 1s 25ms/step - loss: 8025.1753
Epoch 12/600
29/29 [==============================] - 1s 21ms/step - lo

In [78]:
# Data shifts for the testing data.
shifted_x_test = []
for index in range(logs,test_x.shape[0]-1):
    shifted_x_test.append(test_x_scaled[index - logs : index])
shifted_x_test = np.array(shifted_x_test)

print(shifted_x_test.shape)
    

(101, 5, 43)


In [79]:
# Testing the model
# y_test = learning_model(shifted_x_test)

# print(y_test)
test_yAdjusted = test_y[0:len(test_y)-logs]
learning_model.evaluate(x=shifted_x_test, y=test_yAdjusted)
predict = learning_model.predict(shifted_x_test)

4/4 [==============================] - 2s 7ms/step


In [75]:
import seaborn as sns
sns.scatterplot(data=shifted_x_test, x="date", y="precipitation")

c:\Python38\lib\site-packages\seaborn\_oldcore.py:917: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  val in data


ValueError: Could not interpret value `date` for parameter `x`